In [3]:
import numpy as np
import pandas as pd
import requests 
import os
import tensorflow as tf
from tensorflow import keras

In [140]:
import re 

def get_dna_seq(nseq):
  if nseq.find("5'-[") == -1:
    if nseq.find('(') != -1 and nseq.find(')') != -1:
      s1 = nseq.find('(')
      s2 = nseq.rfind(')')
      num = nseq[s2+1]
      if nseq.rfind(')')+2 < len(nseq) and nseq[nseq.rfind(')')+2].isdigit():
        num = int(num) * 10 + int(nseq[nseq.find(')')+2])
      else:
        num = int(num)

      sub = nseq[s1 + 1 : s2]
      sub = sub * num
      nseq = nseq[:s1] + sub + nseq[s2+1:]
    

    return "".join(c for c in nseq if c.isalpha())
  
  
  nseq = nseq[: nseq.rfind("]-3'")]
  while nseq.find('(') != -1 and nseq.find(')') != -1:
    s1 = nseq.find('(')
    s2 = nseq.rfind(')')
    num = nseq[s2+1]
    if nseq.rfind(')')+2 < len(nseq) and nseq[nseq.rfind(')')+2].isdigit():
      num = int(num) * 10 + int(nseq[nseq.find(')')+2])
    else:
      num = int(num)

    sub = nseq[s1 + 1 : s2]
    sub = sub * num
    nseq = nseq[:s1] + sub + nseq[s2+1:]

  

  nseq = nseq[4: ]
  new_seq = ""
  for c in range(len(nseq)):
    if nseq[c].isdigit():
      lit = nseq[c-1]
      new_seq = new_seq + lit * int(nseq[c])
    else:
      lit = nseq[c]
      new_seq = new_seq + lit
  nseq = new_seq    
  nseq="".join(c for c in nseq if c.isalpha())
  return nseq


def get_rna_seq(nseq):

  if nseq == "5'-[CGG]90-3'":
    nseq = "5'-[(CGG)90]-3'"

  if nseq == "5'-[GGGGCC]23-3'":
    nseq = "5'-[(GGGGCC)23]-3'"
  if nseq == "5'-[AGGGAU-3?,5?-AGG*G*AU]-3'":
    return "AGGGAUAGGGAU"
  if nseq == "5'-[UGAGAAGGGCGUAGG]-3' and 5'-[GTCTTAGGGCAG]-3":
    return "UGAGAAGGGCGUAGGGTCTTAGGGCAG"

  if nseq.find("5'-[") == -1:
    if nseq.find('(') != -1 and nseq.find(')') != -1:
      s1 = nseq.find('(')
      s2 = nseq.rfind(')')
      num = nseq[s2+1]
      if nseq.rfind(')')+2 < len(nseq) and nseq[nseq.rfind(')')+2].isdigit():
        num = int(num) * 10 + int(nseq[nseq.find(')')+2])
      else:
        num = int(num)

      sub = nseq[s1 + 1 : s2]
      sub = sub * num
      nseq = nseq[:s1] + sub + nseq[s2+1:]
    

    return "".join(c for c in nseq if c.isalpha())
  
  
  nseq = nseq[: nseq.rfind("]-3'")]
  while nseq.find('(') != -1 and nseq.find(')') != -1:
    s1 = nseq.find('(')
    s2 = nseq.rfind(')')
    num = nseq[s2+1]
    if nseq.rfind(')')+2 < len(nseq) and nseq[nseq.rfind(')')+2].isdigit():
      if num.isdigit():
        num = int(num) * 10 + int(nseq[nseq.find(')')+2])

      elif nseq.rfind(')')+ 3 < len(nseq) and not nseq[nseq.rfind(')')+3].isdigit():
        num = int(nseq[nseq.find(')')+2])
      elif nseq.rfind(')')+ 3 < len(nseq) and nseq[nseq.rfind(')')+3].isdigit():
        num = int(nseq[nseq.find(')')+2]) * 10 + int(nseq[nseq.find(')')+3])
      
    else:
      num = int(num)

    sub = nseq[s1 + 1 : s2]
    sub = sub * num
    nseq = nseq[:s1] + sub + nseq[s2+1:]

  nseq = nseq[4: ]
  new_seq = ""
  
  for c in range(len(nseq)):
    if nseq[c].isdigit():
      lit = nseq[c-1]
      new_seq = new_seq + lit * int(nseq[c])
    else:
      lit = nseq[c]
      if lit.islower():
        lit=''
      
      new_seq = new_seq + lit

  nseq = new_seq    
  nseq="".join(c for c in nseq if c.isalpha())
  return nseq


##DATA

Downloading protein's sequences from G4IPDB http://people.iiti.ac.in/~amitk/bsbe/ipdb/index.php

In [ ]:
%cd /content/

/content


In [ ]:
def prepare_name(name):
  for ch in [' ', '(', '-','?','+', '.']:
    name = name.replace(ch, '_')
    
  for ch in [',' ,')','=']:
    name = name.replace(ch, '')

  return name

In [53]:
!rm -r G4IPDB
!mkdir G4IPDB
!mkdir G4IPDB/DNA
!mkdir G4IPDB/RNA

In [54]:
table = pd.read_html('/content/G4RNAP.html')
new_column = list()
new_rna = list()
for index, row in table[0].iterrows():
    prot_id = row['UniProt-ID']
    id = row['Interaction ID']

    if prot_id and prot_id != 'nan' and prot_id is not np.NaN:
      url = "https://rest.uniprot.org/uniprotkb/" + prot_id + ".fasta"
      r = requests.get(url)
      rna_name = prepare_name(row['RNA Target_Name']) 
      new_rna.append(rna_name) 
      os.mkdir('/content/G4IPDB/RNA/' +id)
      with open('/content/G4IPDB/RNA/' +id +'/'+ prot_id + '.txt', 'w') as f: 
        f.write(str(r.content).split("\\n")[0][2:] + '\n')
        f.write("".join(str(r.content).split("\\n")[1:-1])+ '\n')
      
      new_column.append("".join(str(r.content).split("\\n")[1:-1]))

      with open('/content/G4IPDB/RNA/' +id +'/'+ rna_name + '.txt', 'w') as f:

        f.write(row['RNA Target_Name'] + '\n')
    
    else:
      new_column.append('')
      new_rna.append('')
table[0]['ProtSeq_loc'] = new_column
table[0]['RNA_Target_Name'] = new_rna
RG4IP = table[0].drop(columns='RNA Target_Name')
RG4IP = RG4IP.dropna(subset=['UniProt-ID', 'Target_RNA_Sequence']).reset_index(drop=True)

In [55]:
RG4IP.to_csv('/content/RG4IP.csv')
RG4IP

,Interaction ID,Target_RNA_Sequence,Interacting_Protein_Name,UniProt-ID,UniProt-Entry name,Reference:PMID,ProtSeq_loc,RNA_Target_Name
0,G4RIP1,5'-[UUAGGGUUAGGGUUAGGGUUAGGGUUA]-3',BG4( antibody ),Q13158,FADD_HUMAN,25421962,MDPFLVLLHSVSSSLSSSELTELKFLCLGRVGKRKLERVQSGLDLF...,human_telomeric_RNA__TERRA_G_quadruplex__GQ
1,G4RIP10,5'-[UUAGGGUUAGGGUUAGGGUUAGGGUUA]-3',TRF2,Q15554,TERF2_HUMAN,22780456,MAAGAGTAGPASGPGVVRDPAASQPRKRPGREGGEGARRSDTMAGG...,hTELO_hyb__hybrid_TERRA_hTELO_G_quadruplex
2,G4RIP11,5'-[GAUUGGAAGGGAGGGAGGUG]-3',FMRP ISO1,Q06787,FMR1_HUMAN,22134704,MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPDRQIPFHD...,G_quadruplex_RNA_forming_S3F_Sh_8AP_mRNA
3,G4RIP12,5'-[GAUUGGAAGGGAGGGAGGUG]-3',FMRP ISO2,Q06787,FMR1_HUMAN,22134704,MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPDRQIPFHD...,G_quadruplex_RNA_forming_S3F_Sh_8AP_mRNA
4,G4RIP13,5'-[GAUUGGAAGGGAGGGAGGUG]-3',FMRP ISO3,Q06787,FMR1_HUMAN,22134704,MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPDRQIPFHD...,G_quadruplex_RNA_forming_S3F_Sh_8AP_mRNA
5,G4RIP14,"5'-[ GGGUACGGGAGGGUAAGGC UGUGGGUCGCGUG]-3', (G...",FMRP Isoform 1,Q06787,FMR1_HUMAN,26412477,MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPDRQIPFHD...,G_quadruplexe_NR2B_mRNA
6,G4RIP15,"5'-[GGGUACGGGAGGGUAAGGC UGUGGGUCGCGUG ]-3',(G7...",FMRP S500D,Q06787,FMR1_HUMAN,26412477,MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPDRQIPFHD...,G_quadruplexe_NR2B_mRNA
7,G4RIP16,"5'-[CUGCGAUGUGGAAGGAGUGGCUGGGUUGUGCAGC]-3',(G7...",RGG motif of FMRP(HUMAN),Q06787,FMR1_HUMAN,26374839,MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPDRQIPFHD...,G_quadruplexes_mRNA
11,G4RIP2,5'-[UUAGGGUUAGGGUUAGGGUUAGGGUUA]-3',Cpds(carboxypyridostatin),P52719,PEPS_ASPPH,25421962,MRITSAIASLLLVGTATSLQNPHRRAVPPPLTHRSVASRAVPVERR...,human_telomeric_RNA__TERRA_G_quadruplex__GQ
18,G4RIP26,"5'-[CUGCGAUGUGGAAGGAGUGGCUGGGUUGUGCAGC]-3',(G7...",RGG motif of FMRP(MOUSE),P35922,FMR1_MOUSE,26374839,MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPERQIPFHD...,G_quadruplexes_mRNA


In [56]:
table = pd.read_html('/content/G4dnaP.html')
new_column = list()
new_rna = list()
for index, row in table[0].iterrows():
    prot_id = row['UniProt-ID']
    id = row['Interaction ID']

    if prot_id and prot_id != 'nan' and prot_id is not np.NaN:
      url = "https://rest.uniprot.org/uniprotkb/" + prot_id + ".fasta"
      r = requests.get(url)
      rna_name = prepare_name(row['DNA Target_Name']) 
      new_rna.append(rna_name) 
      os.mkdir('/content/G4IPDB/DNA/' +id)
      with open('/content/G4IPDB/DNA/' +id +'/'+ prot_id + '.txt', 'w') as f: 
        f.write(str(r.content).split("\\n")[0][2:] + '\n')
        f.write("".join(str(r.content).split("\\n")[1:-1])+ '\n')
      
      new_column.append("".join(str(r.content).split("\\n")[1:-1]))

      with open('/content/G4IPDB/DNA/' +id +'/'+ rna_name + '.txt', 'w') as f:

        f.write(row['DNA Target_Name'] + '\n')
    
    else:
      new_column.append('')
      new_rna.append('')
table[0]['ProtSeq_loc'] = new_column
table[0]['DNA_Target_Name'] = new_rna
DG4IP = table[0].drop(columns='DNA Target_Name') 
DG4IP = DG4IP.dropna(subset=['UniProt-ID', 'Target_DNA_Sequence']).reset_index(drop=True)

In [57]:
DG4IP.to_csv('/content/DG4IP.csv')
DG4IP

,Interaction ID,Target_DNA_Sequence,Interacting_Protein_Name,UniProt-ID,UniProt-Entry name,Reference:PMID,ProtSeq_loc,DNA_Target_Name
0,G4DIP1,5'-[TTTTTGGGGACTTTCCAGGGAGGCGTGGCCTGGGCGGGTTTT...,NCL(nucleolin),P19338,NUCL_HUMAN,26354862.0,MVKLAKAGKNQGDPKKMAPPPKEVEEDSEDEEMSEDEEDDSSGEEV...,LTR_II
1,G4DIP10,5'-[TGGCGACGGCAGCGAGGCCGGGCGGGCGCGAGGGAGGGG/iC...,BLM Helicase(Bloom syndrome RecQ like helicase),P54132,BLM_HUMAN,25418155.0,MAAVPQNNLQEQLERHSARTLNNKLSLSKPKFSGFTFKKKTSSDNN...,G4_forming_human_telomere
5,G4DIP103,repetitiveTTAGGG,telomeric repeat binding factor 2,Q15554,TERF2_HUMAN,16302000.0,MAAGAGTAGPASGPGVVRDPAASQPRKRPGREGGEGARRSDTMAGG...,Telomeric_DNA_G_rich_duplex_sequences
6,G4DIP104,5'-[(GGTTAG)5]-3',tumor necrosis factor receptor superfamily mem...,P43489,TNR4_HUMAN,16445281.0,MCVGARRLGRGPCAALLLLGLGLSTVTGLHCVGDTYPSNDRCCHEC...,G_quadruplex_human_telomeric_sequence
7,G4DIP105,5'-[(GGTTAG)5]-3',early growth response 1,P18146,EGR1_HUMAN,16445281.0,MAAAKAEMQLMSPLQISDPFGSFPHSPTMDNYPKLEEMMLLSNGAP...,G_quadruplex_human_telomeric_sequence
...,...,...,...,...,...,...,...,...
123,G4DIP88,5'-[(CATGGGGGCGGGAAGGGGCGGGGTCT)]-3',myogenic factor 5,P13349,MYF5_HUMAN,18511462.0,MDVMDGCQFSPSEYFYDGSCIPSPEGEFGDEFVPRVAAFGAHKAEL...,quadruplex_Integrin_DNA
124,G4DIP89,5'-[(CATGGGGGCGGGAAGGGGCGGGGTCT)]-3',myogenic factor 6,P23409,MYF6_HUMAN,18511462.0,MMMDLFETGSYFFYLDGENVTLQPLEVAEGSPLYPGSDGTLSPCQD...,quadruplex_Integrin_DNA
125,G4DIP90,5'-[(CATGGGGGCGGGAAGGGGCGGGGTCT)]-3',myogenin (myogenic factor 4),P15173,MYOG_HUMAN,18511462.0,MELYETSPYFYQEPRFYDGENYLPVHLQGFEPPGYERTELTLSPEA...,quadruplex_Integrin_DNA
126,G4DIP91,5'-[C3G3CG3CGCGAG3AG4AG2]-3',poly(ADP-ribose) polymerase 1,P09874,PARP1_HUMAN,18511462.0,MAESSDKLYRVEYAKSGRASCKKCSESIPKDSLRMAIMVQSPMFDG...,c_kit_1_quadruplex


107 интеракций в DNA и 77 в RNA..........

но кажется что при составлении датасета возникло некоторое число ошибок

In [ ]:
DG4IP[['Interaction ID', 'Target_DNA_Sequence','DNA_Target_Name']].to_csv('/content/DNAtoverify.csv', index=False)

In [59]:
piece = pd.read_csv('/content/DNAverified2.csv', delimiter=';')
dfd = DG4IP.reset_index(drop=True)
dfd['Target_DNA_Sequence'] = piece['quadruplex_sequence']

In [ ]:
RG4IP[['Interaction ID', 'Target_RNA_Sequence','RNA_Target_Name']].to_csv('/content/RNAtoverify.csv', index=False)

In [144]:
dfd['Target_DNA_Sequence'] = dfd['Target_DNA_Sequence'].apply(get_dna_seq)

In [147]:
dfr = RG4IP.reset_index(drop=True)
dfr['Target_RNA_Sequence'] = dfr['Target_RNA_Sequence'].apply(get_rna_seq)

##1 DL for G4BP
"*Первая deep learning, которая независимо считает эмбеддинги для протеина и квадруплекса, конкатенирует, подает в MLP и получает вероятность связывания*" из великого письма

начнем с эмбедингов протеина


In [ ]:
DG4IP = pd.read_csv('/content/DG4IP.csv')
RG4IP = pd.read_csv('/content/RG4IP.csv')

In [153]:
!git clone https://github.com/hfuulgb/PDBP-Fusion.git

Cloning into 'PDBP-Fusion'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 53 (delta 14), reused 36 (delta 10), pack-reused 0
Unpacking objects: 100% (53/53), 12.55 MiB | 13.82 MiB/s, done.


In [154]:
%cd PDBP-Fusion/

/content/PDBP-Fusion


In [4]:
# -*- coding: utf-8 -*-
import os
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import layers, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score , accuracy_score, precision_score,f1_score,matthews_corrcoef
from tensorflow.keras.layers import BatchNormalization
from keras.layers.core import Dense, Dropout, Flatten,Activation
from keras.layers.convolutional import Convolution1D, MaxPooling1D, SeparableConv1D
from keras.layers import LSTM, Bidirectional,Dense,Embedding,Input,concatenate
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
import numpy as np
import utils.tools as utils
import encode_schema as tool
import cnn_models as cnnmodel
from datetime import datetime


# ================gpu动态占用====================#
import tensorflow as tf
from keras.backend import set_session
config = tf.compat.v1.ConfigProto()
# A "Best-fit with coalescing" algorithm, simplified from
#   a version of dlmalloc.
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))
# =================gpu动态占用====================#


ModuleNotFoundError: ignored

In [158]:
import json
import keras 

config = '{"class_name": "Model", "config": {"name": "DBP_CNN_model", "layers": [{"name": "input", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 600, 20], "dtype": "float32", "sparse": false, "name": "input"}, "inbound_nodes": []}, {"name": "conv1d_10", "class_name": "Conv1D", "config": {"name": "conv1d_10", "trainable": true, "dtype": "float32", "filters": 64, "kernel_size": [9], "strides": [1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1], "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "inbound_nodes": [[["input", 0, 0, {}]]]}, {"name": "max_pooling1d_10", "class_name": "MaxPooling1D", "config": {"name": "max_pooling1d_10", "trainable": true, "dtype": "float32", "strides": [2], "pool_size": [2], "padding": "valid", "data_format": "channels_last"}, "inbound_nodes": [[["conv1d_10", 0, 0, {}]]]}, {"name": "dropout_10", "class_name": "Dropout", "config": {"name": "dropout_10", "trainable": true, "dtype": "float32", "rate": 0.3, "noise_shape": null, "seed": null}, "inbound_nodes": [[["max_pooling1d_10", 0, 0, {}]]]}, {"name": "conv1d_11", "class_name": "Conv1D", "config": {"name": "conv1d_11", "trainable": true, "dtype": "float32", "filters": 64, "kernel_size": [9], "strides": [1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1], "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "inbound_nodes": [[["dropout_10", 0, 0, {}]]]}, {"name": "max_pooling1d_11", "class_name": "MaxPooling1D", "config": {"name": "max_pooling1d_11", "trainable": true, "dtype": "float32", "strides": [2], "pool_size": [2], "padding": "valid", "data_format": "channels_last"}, "inbound_nodes": [[["conv1d_11", 0, 0, {}]]]}, {"name": "dropout_11", "class_name": "Dropout", "config": {"name": "dropout_11", "trainable": true, "dtype": "float32", "rate": 0.3, "noise_shape": null, "seed": null}, "inbound_nodes": [[["max_pooling1d_11", 0, 0, {}]]]}, {"name": "bidirectional_6", "class_name": "Bidirectional", "config": {"name": "bidirectional_6", "trainable": true, "dtype": "float32", "layer": {"class_name": "LSTM", "config": {"name": "lstm_5", "trainable": true, "dtype": "float32", "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "units": 16, "activation": "tanh", "recurrent_activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "recurrent_initializer": {"class_name": "Orthogonal", "config": {"gain": 1.0, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "unit_forget_bias": true, "kernel_regularizer": null, "recurrent_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "recurrent_constraint": null, "bias_constraint": null, "dropout": 0.0, "recurrent_dropout": 0.0, "implementation": 2}}, "merge_mode": "concat"}, "inbound_nodes": [[["dropout_11", 0, 0, {}]]]}, {"name": "dropout_25", "class_name": "Dropout", "config": {"name": "dropout_25", "trainable": true, "dtype": "float32", "rate": 0.3, "noise_shape": null, "seed": null}, "inbound_nodes": [[["bidirectional_6", 0, 0, {}]]]}, {"name": "flatten_6", "class_name": "Flatten", "config": {"name": "flatten_6", "trainable": true, "dtype": "float32", "data_format": "channels_last"}, "inbound_nodes": [[["dropout_25", 0, 0, {}]]]}, {"name": "dense_11", "class_name": "Dense", "config": {"name": "dense_11", "trainable": true, "dtype": "float32", "units": 128, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "inbound_nodes": [[["flatten_6", 0, 0, {}]]]}, {"name": "batch_normalization_5", "class_name": "BatchNormalization", "config": {"name": "batch_normalization_5", "trainable": true, "dtype": "float32", "axis": -1, "momentum": 0.99, "epsilon": 0.001, "center": true, "scale": true, "beta_initializer": {"class_name": "Zeros", "config": {}}, "gamma_initializer": {"class_name": "Ones", "config": {}}, "moving_mean_initializer": {"class_name": "Zeros", "config": {}}, "moving_variance_initializer": {"class_name": "Ones", "config": {}}, "beta_regularizer": null, "gamma_regularizer": null, "beta_constraint": null, "gamma_constraint": null}, "inbound_nodes": [[["dense_11", 0, 0, {}]]]}, {"name": "dropout_3", "class_name": "Dropout", "config": {"name": "dropout_3", "trainable": true, "dtype": "float32", "rate": 0.3, "noise_shape": null, "seed": null}, "inbound_nodes": [[["batch_normalization_5", 0, 0, {}]]]}, {"name": "dense_12", "class_name": "Dense", "config": {"name": "dense_12", "trainable": true, "dtype": "float32", "units": 2, "activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "inbound_nodes": [[["dropout_3", 0, 0, {}]]]}], "input_layers": [["input", 0, 0]], "output_layers": [["dense_12", 0, 0]]}, "keras_version": "2.3.1", "backend": "tensorflow"}'
# Create the model using the configuration 
model = keras.models.model_from_json(config)

# Get the layers of the model 
layers = model.layers


In [ ]:
model2= tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 600, 20)]         0         
                                                                 
 conv1d_10 (Conv1D)          (None, 600, 64)           11584     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 300, 64)          0         
 g1D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 300, 64)           0         
                                                                 
 conv1d_11 (Conv1D)          (None, 300, 64)           36928     
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 150, 64)          0         
 g1D)                                                      

In [ ]:
emb = model2.predict(np.expand_dims(encode_schema.get_seq_concolutional_array(string), 0))

1/1 [==============================] - 0s 48ms/step


In [ ]:
emb.shape

(1, 128)

### embeding for DNA G4. Try DeepG4

In [149]:
%cd /content/

/content


In [5]:
!wget https://github.com/raphaelmourad/DeepG4/raw/master/inst/extdata/DeepG4_classic_rescale_BW_sampling_02_03_2021/2021-03-02T16-17-28Z/best_model.h5

--2023-05-16 21:41:23--  https://github.com/raphaelmourad/DeepG4/raw/master/inst/extdata/DeepG4_classic_rescale_BW_sampling_02_03_2021/2021-03-02T16-17-28Z/best_model.h5
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/raphaelmourad/DeepG4/master/inst/extdata/DeepG4_classic_rescale_BW_sampling_02_03_2021/2021-03-02T16-17-28Z/best_model.h5 [following]
--2023-05-16 21:41:23--  https://raw.githubusercontent.com/raphaelmourad/DeepG4/master/inst/extdata/DeepG4_classic_rescale_BW_sampling_02_03_2021/2021-03-02T16-17-28Z/best_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 759488 (742K) [applic

In [6]:
deepg4 = keras.models.load_model('/content/best_model.h5')

In [ ]:
deepg4.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_input (InputLayer)     [(None, 201, 4)]          0         
                                                                 
 conv_motif (Conv1D)         (None, 182, 500)          40500     
                                                                 
 average_motif_signal (Avera  (None, 11, 500)          0         
 gePooling1D)                                                    
                                                                 
 max_pooling_for_motif (Glob  (None, 500)              0         
 alMaxPooling1D)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 500)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               5010

In [152]:
model_g4= tf.keras.Model(inputs=deepg4.input, outputs=deepg4.layers[-3].output,
                         name='model_G4')
model_g4.summary()

Model: "model_G4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_input (InputLayer)     [(None, 201, 4)]          0         
                                                                 
 conv_motif (Conv1D)         (None, 182, 500)          40500     
                                                                 
 average_motif_signal (Avera  (None, 11, 500)          0         
 gePooling1D)                                                    
                                                                 
 max_pooling_for_motif (Glob  (None, 500)              0         
 alMaxPooling1D)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 500)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               501

эмбединг вроде получили осталось понять как предобработать квадруплекс. а в этом нам поможет наш тайный друг! 

тайный друг не помог, помогли собственные ручки - залезть в исходники, исправить ошибки, написать функцию

In [161]:
dfd.columns

Index(['Interaction ID', 'Target_DNA_Sequence', 'Interacting_Protein_Name',
       'UniProt-ID', 'UniProt-Entry name', 'Reference:PMID', 'ProtSeq_loc',
       'DNA_Target_Name'],
      dtype='object')

In [162]:
dfd.Target_DNA_Sequence.to_csv('/content/G4IPDB/Target_DNA_Sequence.txt', header=False, index=False)

In [163]:
!head /content/G4IPDB/Target_DNA_Sequence.txt

TTTTTGGGGACTTTCCAGGGAGGCGTGGCCTGGGCGGGTTTTT
TGGCGACGGCAGCGAGGCCGGGCGGGCGCGAGGGAGGGGTTTTTTTTTTTTTTTTTTTT
TTAGGGTTAGGGTTAGGGTTAGGGTTAGGG
GGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGGGG
GGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGGGG
GGGTTAGGGTTAGGGTTAGGG
GGTTGGTGTGGTTGG
GGTTGGTGTGGTTGG
TGGACCAGACCTAGCAGCTATGGGGGAGCTGGGGAAGGTGGGAATGTGA
GGGTTAGGGTTAGGGTTAGGGGGG


In [7]:
#edited /content/PDBP-Fusion/predict.py

from keras.models import load_model
import numpy as np
from datetime import datetime
import tensorflow as tf
import numpy as np
import pdb

def get_nuq_concolutional_array(seq, str_len=201, RNA=False):
    alpha = 'ACGT'
    if RNA:
      alpha = 'ACGU'
    row = (len(seq))
    new_array = np.zeros((row, 4))

    for i, val in enumerate(seq):

        if val not in alpha:
            if val == 'Z':
                new_array[i] = np.array([0.0] * 4)
            # if val == 'S':
            #     new_array[i] = np.array([0, 0.5, 0.5, 0, 0])
            continue

        try:
            index = alpha.index(val)
            new_array[i][index] = 1
        except ValueError:
            pdb.set_trace()
    return new_array  

def get_seq_concolutional_array(seq): #from encode_schema.py
    # seq = seq.replace('U', 'T')
    alpha = 'ACDEFGHIKLMNPQRSTVWY'
    row = (len(seq))
    new_array = np.zeros((row, 20))

    for i, val in enumerate(seq):

        if val not in 'ACDEFGHIKLMNPQRSTVWY':
            if val == 'Z':
                new_array[i] = np.array([0.0] * 20)
            # if val == 'S':
            #     new_array[i] = np.array([0, 0.5, 0.5, 0, 0])
            continue

        try:
            index = alpha.index(val)
            new_array[i][index] = 1
        except ValueError:
            pdb.set_trace()
    return new_array

def read_seq_onehot(seq_file, str_len=600): #from encode_schema.py fix const 600 (was 850)
    seq_list = []
    seq = ''
    with open(seq_file, 'r') as fp:
        index = 0
        for line in fp:
            index += 1
            string = line
            if len(line)>=str_len:
                string=string[:str_len]
            else:
                string=string+'Z'*(str_len-len(string))
            
            assert len(string)==str_len
            line = string

            seq = line
            seq_array = get_seq_concolutional_array(seq)
            seq_list.append(seq_array)
        print(index)
    return np.array(seq_list, dtype=object)

def read_nuq_onehot(seq_file, str_len=201, RNA=False): 
    seq_list = []
    seq = ''
    with open(seq_file, 'r') as fp:
        index = 0
        for line in fp:
            index += 1
            string = line
            if len(line)>=str_len:
                string=string[:str_len]
            else:
                string=string+'Z'*(str_len-len(string))
            
            assert len(string)==str_len
            line = string

            seq = line
            seq_array = get_nuq_concolutional_array(seq,str_len=201, RNA=RNA)
            seq_list.append(seq_array)
        print(index)
    return np.array(seq_list, dtype=object)


class Pred():
    def __init__(self,model_path='./model_save/model_'):
        self.item_model_path = '/content/PDBP-Fusion/model_save/model_0.hdf5'
        #self.model = load_model(item_model_path)

    def sample_predict1(self,string,str_len=600):
        # make sure the input sequnce less than 850, otherwise drop string when length excell 850
        if len(string)>=str_len:
            string=string[:str_len]
        else:
            string=string+'Z'*(str_len-len(string))
        #print('len: ',str_len)
        assert len(string)==str_len

        #向量化
        #print(get_seq_concolutional_array(string)[:5,:])
        string_vector=get_seq_concolutional_array(string)
        string_vector=np.expand_dims(string_vector,0)

        #load model and predict
        result=[]
        self.model = load_model(self.item_model_path)
        predict=self.model.predict(string_vector)[0]
        print('prediction is {}'.format(predict))
        result.append(predict)
        print(result)

        return result

    def sample_predict(self,string,str_len=600):
        # make sure the input sequnce less than 850, otherwise drop string when length excell 850

        seq = read_seq_onehot(string)
        print(seq.shape)
        # seq = np.asarray(seq).astype(np.float32)
        # string_vector=get_seq_concolutional_array(string)
        # string_vector=np.expand_dims(string_vector,0)

        #load model and predict
        result=[]
        self.model = load_model(self.item_model_path)
        predict=self.model.predict(tf.convert_to_tensor(seq, dtype=tf.float32))
        print('prediction is {}'.format(predict))
        result.append(predict)

        return result

    def get_emb_dna(self, path, str_len=201):
        seq = read_nuq_onehot(path, str_len=201)
        print(seq.shape)
        result = []
        self.model = tf.keras.Model(inputs=deepg4.input, outputs=deepg4.layers[-3].output,
                         name='model_EmbG4')
        predict = self.model.predict(tf.convert_to_tensor(seq, dtype=tf.float32))
        return predict

    def get_emb_rna(self, path, str_len=201):
        seq = read_nuq_onehot(path, str_len=201, RNA=True)
        print(seq.shape)
        result = []
        self.model = tf.keras.Model(inputs=deepg4.input, outputs=deepg4.layers[-3].output,
                         name='model_EmbG4')
        predict = self.model.predict(tf.convert_to_tensor(seq, dtype=tf.float32))
        return predict
Pr=Pred()
start = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

dna_g4_emb = Pr.get_emb_dna('/content/G4IPDB/Target_DNA_Sequence.txt') #this is our embedings for DNA Target Seq
print('')
end = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
print('start: %s' % start)
print('end: %s' % end)

# classes = np.argmax(ans[0], axis=1)
# print('DBPs = ', np.sum(classes), ' nonDBPs = ', len(classes) - np.sum(classes))

107
(107, 201, 4)
4/4 [==============================] - 0s 8ms/step

start: 2023-05-16-21:41:49
end: 2023-05-16-21:41:49


In [177]:
dfr.Target_RNA_Sequence.to_csv('/content/G4IPDB/Target_RNA_Sequence.txt', header=False, index=False)

In [9]:
Pr=Pred()
start = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

rna_g4_emb = Pr.get_emb_rna('/content/G4IPDB/Target_RNA_Sequence.txt') #this is our embedings for DNA Target Seq
print('')
end = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
print('start: %s' % start)
print('end: %s' % end)


60
(60, 201, 4)
2/2 [==============================] - 0s 11ms/step

start: 2023-05-16-21:42:25
end: 2023-05-16-21:42:25
